## PREPARATION DES DONNEES POUR ANALYSE ET MODELISATION

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk 
import datetime
import re


### Importation des données

In [25]:
DEPT_CODE = 69  # à changer facilement ici

# --- Chargement des fichiers ---
path_existants = f"../data/donnees_dpe_existants_{DEPT_CODE}.csv"
path_neufs = f"../data/donnees_dpe_neufs_{DEPT_CODE}.csv"

logements_existants = pd.read_csv(path_existants)
logements_neufs = pd.read_csv(path_neufs)

print(f"✅ Chargé DPE {DEPT_CODE} : {logements_existants.shape[0]} existants, {logements_neufs.shape[0]} neufs")

C:\Users\ychen\AppData\Local\Temp\ipykernel_20916\3552548052.py:7: DtypeWarning: Columns (7,26,39,51,161,162,163,164,169,170,171,172,174,175,176,177,213) have mixed types. Specify dtype option on import or set low_memory=False.
  logements_existants = pd.read_csv(path_existants)
C:\Users\ychen\AppData\Local\Temp\ipykernel_20916\3552548052.py:8: DtypeWarning: Columns (7,26,31,37,49,51,136,137,138,139,140,143,144,145,147,148,149,151,152,153,154,157,158,159,161,162,163,166,169,170,171,175,177,178,179,180,191,194,195,199) have mixed types. Specify dtype option on import or set low_memory=False.
  logements_neufs = pd.read_csv(path_neufs)


✅ Chargé DPE 69 : 430668 existants, 40277 neufs


In [26]:
# Ajouter de la colonne "Logement" dans chaque DF
logements_existants["Logement"]="Ancien"
logements_neufs["Logement"]="Neuf"

In [27]:
# Ajout année de construction
now=datetime.datetime.now()
logements_neufs["annee_construction"]=now.year
print (logements_neufs["annee_construction"].value_counts())

annee_construction
2025    40277
Name: count, dtype: int64


In [28]:
# Fusionner les deux DF
logements_merged=pd.concat([logements_neufs,logements_existants], join = 'inner')
print(logements_merged.columns)
print(len(logements_merged.columns))

Index(['numero_dpe', 'date_derniere_modification_dpe',
       'date_visite_diagnostiqueur', 'date_etablissement_dpe',
       'date_reception_dpe', 'date_fin_validite_dpe', 'numero_dpe_remplace',
       'numero_dpe_immeuble_associe', 'id_rnb', 'provenance_id_rnb',
       ...
       'surface_totale_capteurs_pv', 'nombre_module',
       'production_electricite_pv_kwhep_par_an',
       'electricite_pv_autoconsommee', '_geopoint', '_id', '_i', '_rand',
       'Logement', 'annee_construction'],
      dtype='object', length=209)
209


In [29]:
# Créer une colonne avec uniquement l'année de la Date de réception du DPE
logements_merged["annee_reception_DPE"]=logements_merged["date_reception_dpe"].str[:4]
print(logements_merged[["annee_reception_DPE","date_reception_dpe"]])

       annee_reception_DPE date_reception_dpe
0                     2021         2021-07-01
1                     2021         2021-07-05
2                     2021         2021-07-05
3                     2021         2021-07-05
4                     2021         2021-07-05
...                    ...                ...
430663                2025         2025-10-27
430664                2025         2025-10-27
430665                2025         2025-10-27
430666                2025         2025-10-27
430667                2025         2025-10-27

[470945 rows x 2 columns]


C:\Users\ychen\AppData\Local\Temp\ipykernel_20916\2634798736.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  logements_merged["annee_reception_DPE"]=logements_merged["date_reception_dpe"].str[:4]


In [30]:
# Créer une colonne periode_construction avec ces classes ci-dessous
def periode_construction(année):
    if année < 1960:
        return "Avant 1960"
    elif 1960 <= année <= 1970:
        return "1961 - 1970" 
    elif 1971 <= année <= 1980:
        return "1971 - 1980" 
    elif 1981 <= année <= 1990:
        return "1981 - 1990"
    elif 1991 <= année <= 2000:
        return "1991 - 2000"
    elif 2001 <= année <= 2010:
        return "2001 - 2010"
    elif pd.isna(année):
        return "Année inconnue"
    else:
        return "Après 2010"

logements_merged["periode_construction"]=logements_merged["annee_construction"].apply(periode_construction)
print(logements_merged[["periode_construction","annee_construction"]])

       periode_construction  annee_construction
0                Après 2010              2025.0
1                Après 2010              2025.0
2                Après 2010              2025.0
3                Après 2010              2025.0
4                Après 2010              2025.0
...                     ...                 ...
430663       Année inconnue                 NaN
430664          1961 - 1970              1962.0
430665          1961 - 1970              1967.0
430666          1961 - 1970              1962.0
430667          1981 - 1990              1986.0

[470945 rows x 2 columns]


C:\Users\ychen\AppData\Local\Temp\ipykernel_20916\1581620481.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  logements_merged["periode_construction"]=logements_merged["annee_construction"].apply(periode_construction)


In [31]:
pd.set_option('display.max_columns', None)  # Affiche toutes les colonnes
logements_merged.head(10)

,numero_dpe,date_derniere_modification_dpe,date_visite_diagnostiqueur,date_etablissement_dpe,date_reception_dpe,date_fin_validite_dpe,numero_dpe_remplace,numero_dpe_immeuble_associe,id_rnb,provenance_id_rnb,numero_rpls_logement,numero_immatriculation_copropriete,modele_dpe,version_dpe,methode_application_dpe,etiquette_dpe,etiquette_ges,classe_altitude,zone_climatique,type_batiment,type_installation_chauffage,type_installation_ecs,hauteur_sous_plafond,nombre_appartement,nombre_niveau_immeuble,nombre_niveau_logement,typologie_logement,appartement_non_visite,surface_habitable_immeuble,surface_habitable_logement,surface_tertiaire_immeuble,classe_inertie_batiment,adresse_ban,numero_voie_ban,nom_rue_ban,nom_commune_ban,code_postal_ban,code_insee_ban,code_departement_ban,code_region_ban,identifiant_ban,coordonnee_cartographique_x_ban,coordonnee_cartographique_y_ban,score_ban,statut_geocodage,adresse_brut,nom_commune_brut,code_postal_brut,numero_etage_appartement,position_logement_dans_immeuble,nom_residence,complement_adresse_batiment,complement_adresse_logement,indicateur_confort_ete,protection_solaire_exterieure,logement_traversant,presence_brasseur_air,inertie_lourde,isolation_toiture,deperditions_enveloppe,deperditions_ponts_thermiques,deperditions_murs,deperditions_planchers_hauts,deperditions_planchers_bas,deperditions_portes,deperditions_baies_vitrees,deperditions_renouvellement_air,qualite_isolation_enveloppe,qualite_isolation_murs,qualite_isolation_plancher_haut_comble_amenage,qualite_isolation_plancher_bas,qualite_isolation_menuiseries,ubat_w_par_m2_k,besoin_chauffage,besoin_ecs,besoin_refroidissement,conso_5_usages_ep,conso_5_usages_par_m2_ep,conso_chauffage_ep,conso_ecs_ep,conso_refroidissement_ep,conso_eclairage_ep,conso_auxiliaires_ep,conso_5_usages_ef,conso_5_usages_par_m2_ef,conso_chauffage_ef,conso_ecs_ef,conso_refroidissement_ef,conso_eclairage_ef,conso_auxiliaires_ef,emission_ges_5_usages,emission_ges_5_usages_par_m2,emission_ges_chauffage,emission_ges_ecs,emission_ges_refroidissement,emission_ges_eclairage,emission_ges_auxiliaires,type_energie_n1,conso_5_usages_ef_energie_n1,conso_chauffage_ef_energie_n1,conso_ecs_ef_energie_n1,cout_total_5_usages_energie_n1,cout_chauffage_energie_n1,cout_ecs_energie_n1,emission_ges_5_usages_energie_n1,emission_ges_chauffage_energie_n1,emission_ges_ecs_energie_n1,type_energie_n2,conso_5_usages_ef_energie_n2,conso_chauffage_ef_energie_n2,conso_ecs_ef_energie_n2,cout_total_5_usages_energie_n2,cout_chauffage_energie_n2,cout_ecs_energie_n2,emission_ges_5_usages_energie_n2,emission_ges_chauffage_energie_n2,emission_ges_ecs_energie_n2,type_energie_n3,conso_5_usages_ef_energie_n3,conso_chauffage_ef_energie_n3,conso_ecs_ef_energie_n3,cout_total_5_usages_energie_n3,cout_chauffage_energie_n3,cout_ecs_energie_n3,emission_ges_5_usages_energie_n3,emission_ges_chauffage_energie_n3,emission_ges_ecs_energie_n3,cout_total_5_usages,cout_chauffage,cout_ecs,cout_refroidissement,cout_eclairage,cout_auxiliaires,type_energie_principale_chauffage,type_generateur_chauffage_principal,type_installation_chauffage_n1,type_emetteur_installation_chauffage_n1,configuration_installation_chauffage_n1,description_installation_chauffage_n1,conso_chauffage_installation_chauffage_n1,surface_chauffee_installation_chauffage_n1,type_generateur_n1_installation_n1,type_energie_generateur_n1_installation_n1,usage_generateur_n1_installation_n1,conso_chauffage_generateur_n1_installation_n1,type_generateur_n2_installation_n1,type_energie_generateur_n2_installation_n1,usage_generateur_n2_installation_n1,conso_chauffage_generateur_n2_installation_n1,type_installation_chauffage_n2,type_emetteur_installation_chauffage_n2,configuration_installation_chauffage_n2,description_installation_chauffage_n2,conso_chauffage_installation_chauffage_n2,surface_chauffee_installation_chauffage_n2,type_generateur_n1_installation_n2,type_energie_generateur_n1_installation_n2,usage_generateur_n1_installation_n2,conso_chauffage_generateur_n1_installa

Gestion des valeurs manquantes

In [32]:
# Identifier et supprimer les colonnes entièrement vides
empty_cols = logements_merged.columns[logements_merged.isna().all()]
logements_merged = logements_merged.drop(columns=empty_cols)

# Supprimer colonnes inutiles
cols_to_drop = ["_geopoint", "_id", "_rand", "_i"]
existing_cols = [col for col in cols_to_drop if col in logements_merged.columns]
logements_merged = logements_merged.drop(columns=existing_cols)

print(f"✅ {len(empty_cols)} colonnes vides supprimées.")
print(f"✅ Colonnes inutiles supprimées : {existing_cols}")
print(f"📊 Nouvelle dimension du DF : {logements_merged.shape}")

✅ 7 colonnes vides supprimées.
✅ Colonnes inutiles supprimées : ['_geopoint', '_id', '_rand', '_i']
📊 Nouvelle dimension du DF : (470945, 200)


In [33]:
# Pourcentage de valeurs manquantes par colonne
missing_ratio = logements_merged.isna().mean().sort_values(ascending=False)
print(missing_ratio.head(100))  # affiche les 20 colonnes les plus vides

numero_rpls_logement                             0.999904
conso_chauffage_generateur_n2_installation_n2    0.998299
type_energie_generateur_n2_installation_n2       0.998161
usage_generateur_n2_installation_n2              0.998161
type_generateur_n2_installation_n2               0.998161
                                                   ...   
type_generateur_chauffage_principal_ecs          0.109238
conso_chauffage_generateur_n1_installation_n1    0.108314
conso_chauffage_installation_chauffage_n1        0.107320
usage_generateur_n1_installation_n1              0.107308
configuration_installation_chauffage_n1          0.107308
Length: 100, dtype: float64


In [34]:
print("📊 Taux moyen de valeurs manquantes avant nettoyage :")
print(f"{logements_merged.isna().mean().mean():.2%}")

# Sélection des colonnes à supprimer (plus de 80% de valeurs manquantes)
cols_to_drop = missing_ratio[missing_ratio > 0.5].index
print(f"📉 Colonnes à supprimer (>{50}% de valeurs manquantes) : {len(cols_to_drop)}")

# Suppression de ces colonnes
logements_merged = logements_merged.drop(columns=cols_to_drop)
print(f"✅ Colonnes supprimées : {len(cols_to_drop)}")
print(f"📊 Nouvelle dimension du DF : {logements_merged.shape}")

print("📊 Taux moyen de valeurs manquantes après nettoyage :")
print(f"{logements_merged.isna().mean().mean():.2%}")

📊 Taux moyen de valeurs manquantes avant nettoyage :
31.91%
📉 Colonnes à supprimer (>50% de valeurs manquantes) : 57
✅ Colonnes supprimées : 57
📊 Nouvelle dimension du DF : (470945, 143)
📊 Taux moyen de valeurs manquantes après nettoyage :
8.20%


In [35]:
# --- 💾 Sauvegarde du jeu de données nettoyé ---
output_path = f"../data/donnees_dpe_{DEPT_CODE}_clean.csv"
logements_merged.to_csv(output_path, index=False)

print("\n✅ Données nettoyées sauvegardées avec succès.")
print(f"📁 Fichier exporté : {output_path}")

# --- 🔍 Vérification rapide ---
print("\nAperçu final :")
display(logements_merged.head(3))

print("\nTypes de données :")
print(logements_merged.dtypes.head(10))

print("\nTaux de valeurs manquantes par colonne (top 10) :")
print(logements_merged.isna().mean().sort_values(ascending=False).head(10))



✅ Données nettoyées sauvegardées avec succès.
📁 Fichier exporté : ../data/donnees_dpe_69_clean.csv

Aperçu final :


,numero_dpe,date_derniere_modification_dpe,date_visite_diagnostiqueur,date_etablissement_dpe,date_reception_dpe,date_fin_validite_dpe,modele_dpe,version_dpe,methode_application_dpe,etiquette_dpe,etiquette_ges,classe_altitude,zone_climatique,type_batiment,type_installation_chauffage,type_installation_ecs,hauteur_sous_plafond,nombre_appartement,nombre_niveau_logement,surface_habitable_immeuble,surface_habitable_logement,classe_inertie_batiment,adresse_ban,numero_voie_ban,nom_rue_ban,nom_commune_ban,code_postal_ban,code_insee_ban,code_departement_ban,code_region_ban,identifiant_ban,coordonnee_cartographique_x_ban,coordonnee_cartographique_y_ban,score_ban,statut_geocodage,adresse_brut,nom_commune_brut,code_postal_brut,numero_etage_appartement,complement_adresse_batiment,complement_adresse_logement,indicateur_confort_ete,protection_solaire_exterieure,logement_traversant,presence_brasseur_air,inertie_lourde,isolation_toiture,deperditions_enveloppe,deperditions_ponts_thermiques,deperditions_murs,deperditions_planchers_hauts,deperditions_planchers_bas,deperditions_portes,deperditions_baies_vitrees,deperditions_renouvellement_air,qualite_isolation_enveloppe,qualite_isolation_murs,qualite_isolation_plancher_bas,qualite_isolation_menuiseries,ubat_w_par_m2_k,besoin_chauffage,besoin_ecs,besoin_refroidissement,conso_5_usages_ep,conso_5_usages_par_m2_ep,conso_chauffage_ep,conso_ecs_ep,conso_refroidissement_ep,conso_eclairage_ep,conso_auxiliaires_ep,conso_5_usages_ef,conso_5_usages_par_m2_ef,conso_chauffage_ef,conso_ecs_ef,conso_refroidissement_ef,conso_eclairage_ef,conso_auxiliaires_ef,emission_ges_5_usages,emission_ges_5_usages_par_m2,emission_ges_chauffage,emission_ges_ecs,emission_ges_refroidissement,emission_ges_eclairage,emission_ges_auxiliaires,type_energie_n1,conso_5_usages_ef_energie_n1,conso_chauffage_ef_energie_n1,conso_ecs_ef_energie_n1,cout_total_5_usages_energie_n1,cout_chauffage_energie_n1,cout_ecs_energie_n1,emission_ges_5_usages_energie_n1,emission_ges_chauffage_energie_n1,emission_ges_ecs_energie_n1,type_energie_n2,conso_5_usages_ef_energie_n2,conso_chauffage_ef_energie_n2,conso_ecs_ef_energie_n2,cout_total_5_usages_energie_n2,cout_chauffage_energie_n2,cout_ecs_energie_n2,emission_ges_5_usages_energie_n2,emission_ges_chauffage_energie_n2,emission_ges_ecs_energie_n2,cout_total_5_usages,cout_chauffage,cout_ecs,cout_refroidissement,cout_eclairage,cout_auxiliaires,type_energie_principale_chauffage,type_generateur_chauffage_principal,type_installation_chauffage_n1,type_emetteur_installation_chauffage_n1,configuration_installation_chauffage_n1,description_installation_chauffage_n1,conso_chauffage_installation_chauffage_n1,surface_chauffee_installation_chauffage_n1,type_generateur_n1_installation_n1,type_energie_generateur_n1_installation_n1,usage_generateur_n1_installation_n1,conso_chauffage_generateur_n1_installation_n1,type_energie_principale_ecs,type_generateur_chauffage_principal_ecs,type_installation_ecs_n1,configuration_installation_ecs_n1,description_installation_ecs_n1,conso_ef_installation_ecs_n1,nombre_logements_desservis_par_installation_ecs_n1,surface_habitable_desservie_par_installation_ecs_n1,type_installation_solaire_n1,type_generateur_n1_ecs_n1,type_energie_generateur_n1_ecs_n1,usage_generateur_n1_ecs_n1,description_generateur_n1_ecs_n1,volume_stockage_generateur_n1_ecs_n1,conso_ef_generateur_n1_ecs_n1,ventilation_posterieure_2012,production_electricite_pv_kwhep_par_an,Logement,annee_construction,annee_reception_DPE,periode_construction
0,2169N0340811D,2023-10-26,2021-06-30,2021-06-30,2021-07-01,2031-06-29,DPE NEUF logement : RT2012,1.0,dpe issu d'une étude thermique réglementaire R...,B,A,inférieur à 400m,H1c,maison,NaN,NaN,2.5,NaN,1.0,NaN,101.6,NaN,La Picolette 69510 Yzeron,NaN,NaN,Yzeron,69510,69269,69.0,84.0,69269_dt014w,826468.95,6512440.36,0.85,adresse géocodée ban à l'adresse,La Picolette,THURINS,69510,NaN,NaN,NaN,moyen,1.0,0.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,très bonne,très bonne,très bonne,insu


Types de données :
numero_dpe                         object
date_derniere_modification_dpe     object
date_visite_diagnostiqueur         object
date_etablissement_dpe             object
date_reception_dpe                 object
date_fin_validite_dpe              object
modele_dpe                         object
version_dpe                       float64
methode_application_dpe            object
etiquette_dpe                      object
dtype: object

Taux de valeurs manquantes par colonne (top 10) :
surface_habitable_immeuble        0.443022
inertie_lourde                    0.436110
isolation_toiture                 0.432099
presence_brasseur_air             0.428797
protection_solaire_exterieure     0.428797
logement_traversant               0.428797
indicateur_confort_ete            0.426942
annee_construction                0.390712
cout_chauffage_energie_n2         0.310480
cout_total_5_usages_energie_n2    0.310480
dtype: float64
